# Введение в MapReduce модель на Python


In [1]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [2]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [3]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [4]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [5]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [6]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [7]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [8]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [9]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [10]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [11]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [12]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [13]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [14]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [15]:
from typing import Iterator
from typing import Tuple
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, np.float64(2.149835093918524)),
 (1, np.float64(2.149835093918524)),
 (2, np.float64(2.149835093918524)),
 (3, np.float64(2.149835093918524)),
 (4, np.float64(2.149835093918524))]

## Inverted index

In [45]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('it', ['0', '1', '2']),
 ('is', ['0', '1', '2']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [17]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [18]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [19]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('is', 18), ('what', 10)]),
 (1, [('a', 2), ('banana', 2), ('it', 18)])]

## TeraSort

In [20]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, np.float64(0.030572911145332005)),
   (None, np.float64(0.036062158403462496)),
   (None, np.float64(0.04457723347824638)),
   (None, np.float64(0.046890279064849105)),
   (None, np.float64(0.07804023876263311)),
   (None, np.float64(0.10955808998649152)),
   (None, np.float64(0.1522539311119635)),
   (None, np.float64(0.17581622124248597)),
   (None, np.float64(0.23519190649320398)),
   (None, np.float64(0.2594412034999731)),
   (None, np.float64(0.27380707093181766)),
   (None, np.float64(0.2761822931299619)),
   (None, np.float64(0.31280897844810296)),
   (None, np.float64(0.3390027622845041)),
   (None, np.float64(0.3468146881085399)),
   (None, np.float64(0.40712234481009835)),
   (None, np.float64(0.42115975982117193)),
   (None, np.float64(0.4246875121378991))]),
 (1,
  [(None, np.float64(0.5267993605172963)),
   (None, np.float64(0.5966749597334741)),
   (None, np.float64(0.6296925107522219)),
   (None, np.float64(0.6894800436829632)),
   (None, np.float64(0.7029

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [21]:
vec1 = [0,   1,   2,   3,   4,   5]
vec2 = [4,   3,   2,   1,   0,  -1]
vec3 = [0.5, 1.5, 2.5, 3.6, 4.7, 5.1]

def MAP(idx, elem):
    i,j = idx
    yield i, elem

def REDUCE(i:int, products:Iterator[list]):
  max = products[0]
  for p in products:
    if max < p:
        max = p
  yield i,max

def RECORDREADER():
  for i in range(3):
    for j in range(len(vec1)):
        yield ((i,j), (vec1,vec2,vec3)[i][j])

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 5), (1, 4), (2, 5.1)]

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [22]:
vec1 = [0,   1,   2,   3,   4,   5]
vec2 = [4,   3,   2,   1,   0,  -1]
vec3 = [0.5, 1.5, 2.5, 3.6, 4.7, 5.1]

def MAP(idx, elem):
    i,j = idx
    yield i, elem

def REDUCE(i:int, products:Iterator[list]):
  s = 0
  for p in products:
    s+=p
  yield i, s/len(products)

def RECORDREADER():
  for i in range(3):
    for j in range(len(vec1)):
        yield ((i,j), (vec1,vec2,vec3)[i][j])

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 2.5), (1, 1.5), (2, 2.983333333333333)]

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [23]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [24]:
def groupbykey(iterable):
    # Сортируем входные данные по ключу
    sorted_iterable = sorted(iterable, key=lambda x: x[0])

    output = dict()

    for (key, val) in sorted_iterable:
        # Добавляем значение к соответствующему ключу
        if key not in output:
            output[key] = list()
        output[key].append(val)

    return output


input_collection = [
    User(id=0, age=55, gender='female', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
print(map_output)
groupbykey(map_output)

[(55, User(id=0, age=55, social_contacts=20, gender='female')), (25, User(id=1, age=25, social_contacts=240, gender='female')), (25, User(id=2, age=25, social_contacts=500, gender='female')), (33, User(id=3, age=33, social_contacts=800, gender='female'))]


{25: [User(id=1, age=25, social_contacts=240, gender='female'),
  User(id=2, age=25, social_contacts=500, gender='female')],
 33: [User(id=3, age=33, social_contacts=800, gender='female')],
 55: [User(id=0, age=55, social_contacts=20, gender='female')]}

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [25]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)
  return reduce_outputs

In [26]:
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d2, d3, d3, d3]


maps = 2
reducers = 3

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        if line.strip():
            yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split():
    if word.strip():
        yield (word, 0)

def REDUCE(key:str, values):
  yield key

# try to set COMBINER=REDUCER and look at the number of values sent over the network
print(list(list(INPUTFORMAT())[1]))
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

[('0:1', 'it is a banana'), ('1:1', 'it is a banana'), ('2:1', 'it is a banana')]


[(0, ['a', 'is', 'it']), (1, ['banana']), (2, ['what'])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [27]:
vec1 = (0,   1,   2,   3,   4,   5)
vec2 = (4,   3,   2,   1,   0,  -1)
vec3 = (0.5, 1.5, 2.5, 3.6, 4.7, 5.1)


def bool_func(elems):
    return bool(np.sum(elems)/len(elems)>2)

def map_el(t):
    dc = dict()
    for elem in t:
        if bool_func(elem):
            dc[elem] = elem
    return dc

def reduce(mapped_t):
    output = list()
    for elem in mapped_t:
        for val in elem:
            output.append(val)
    return output

def record_reader():
  return [vec1,vec2,vec3]




print("Кортежи, среднее значение которых > 2")
output = reduce(list(map(map_el, [record_reader()])))
output = list(output)
output

Кортежи, среднее значение которых > 2


[(0, 1, 2, 3, 4, 5), (0.5, 1.5, 2.5, 3.6, 4.7, 5.1)]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [28]:
vec1 = (0,   1,   2,   3,   4,   5)
vec2 = (4,   3,   2,   1,   0,  -1)
vec3 = (0.5, 1.5, 2.5, 3.6, 4.7, 5.1)
S = set([0,4,5])

def exclude_vals(t):
    return tuple([el for el in t if el in S])

def map_el(t):
    ex_t = exclude_vals(t)
    return (ex_t, ex_t)

def reduce(key, value):
    return (key, key)

def record_reader():
    return [vec1,vec2,vec3]

def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return list(t.items())


output = map(lambda x: reduce(*x), groupbykey(map(map_el, record_reader())))
output = list(output)
output

[((0, 4, 5), (0, 4, 5)), ((4, 0), (4, 0)), ((), ())]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [29]:
import random

def map_el(t):
    return (t, t)

def reduce(key, value):
    return (key, key)

def record_reader():
    return [(random.randint(0, 4), random.randint(0, 4), random.randint(0, 4)) for i in range(20)]

def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return list(t.items())


output = map(lambda x: reduce(*x), groupbykey(map(map_el, record_reader())))
output = list(output)
output

[((3, 1, 1), (3, 1, 1)),
 ((3, 3, 1), (3, 3, 1)),
 ((3, 4, 0), (3, 4, 0)),
 ((2, 4, 0), (2, 4, 0)),
 ((1, 0, 4), (1, 0, 4)),
 ((1, 2, 4), (1, 2, 4)),
 ((3, 1, 2), (3, 1, 2)),
 ((2, 0, 1), (2, 0, 1)),
 ((4, 3, 3), (4, 3, 3)),
 ((1, 3, 0), (1, 3, 0)),
 ((3, 4, 4), (3, 4, 4)),
 ((2, 0, 4), (2, 0, 4)),
 ((4, 0, 0), (4, 0, 0)),
 ((2, 2, 0), (2, 2, 0)),
 ((4, 1, 1), (4, 1, 1)),
 ((0, 0, 1), (0, 0, 1)),
 ((1, 4, 1), (1, 4, 1)),
 ((2, 3, 1), (2, 3, 1)),
 ((3, 1, 3), (3, 1, 3))]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [30]:
def map_el(t):
    return (t, t)

def reduce(key, value):
    if len(value)==2:
        return (key, key)

def record_reader():
    return [(random.randint(0, 4), random.randint(0, 4), random.randint(0, 4)) for i in range(20)]

def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return list(t.items())


output = map(lambda x: reduce(*x), groupbykey(map(map_el, record_reader())))
output = list(output)
output

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 ((1, 4, 0), (1, 4, 0)),
 None,
 None,
 None]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [31]:
def map_el(t, cls):
    return (t, cls)

def reduce(key, value):
    if value == 'R':
        return (key, key)

def record_reader():
    return [((random.randint(0, 4), random.randint(0, 4), random.randint(0, 4)), 'R' if random.random()<0.5 else 'S') for i in range(20)]


output = map(lambda x: reduce(*x), list(map(lambda y: map_el(*y), record_reader())))
output = list(output)
output

[((1, 4, 3), (1, 4, 3)),
 ((0, 0, 0), (0, 0, 0)),
 ((2, 2, 3), (2, 2, 3)),
 ((2, 2, 4), (2, 2, 4)),
 None,
 ((3, 0, 4), (3, 0, 4)),
 ((3, 3, 0), (3, 3, 0)),
 None,
 ((0, 4, 1), (0, 4, 1)),
 None,
 ((1, 0, 4), (1, 0, 4)),
 None,
 ((4, 4, 3), (4, 4, 3)),
 ((4, 0, 4), (4, 0, 4)),
 None,
 None,
 None,
 None,
 ((1, 4, 2), (1, 4, 2)),
 None]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [32]:
def map_el(t,cls):
    if cls == 'R':
        return (t[1], ('R',t[0]))
    elif cls == 'S':
        return (t[0], ('S',t[1]))
    else:
        pass

def reduce(key, values):
    output = list()
    for val in values:
        output.append((val[0],key,val[1]))
    return output

def record_reader():
    return [((random.randint(0, 4), random.randint(0, 4)), 'R' if random.random()<0.5 else 'S') for i in range(5)]

def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return list(t.items())


# Генерация записей
records = record_reader()

# Применение MAP
map_out = list(map(lambda x: map_el(*x), records))
print("MAP:", map_out)

# Shuffle (группировка по ключу)
shuffle_out = groupbykey(map_out)
print("Shuffle:", shuffle_out)

# Применение REDUCE
reduce_out = [reduce(k, v) for k, v in shuffle_out]
print("Reduce:", reduce_out)

MAP: [(0, ('R', 1)), (0, ('R', 3)), (0, ('R', 4)), (3, ('S', 3)), (0, ('S', 3))]
Shuffle: [(0, [('R', 1), ('R', 3), ('R', 4), ('S', 3)]), (3, [('S', 3)])]
Reduce: [[('R', 0, 1), ('R', 0, 3), ('R', 0, 4), ('S', 0, 3)], [('S', 3, 3)]]


### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [47]:
def map_el(t):
    a,b,c = t
    return (a,b)

def agg_oper(elems):
    return sum(elems)

def reduce(key, values):
    return (key,agg_oper(values))

def record_reader():
    return [(random.randint(0, 4), random.randint(0, 4), (random.randint(0, 4))) for i in range(25)]

def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return list(t.items())

output = map(lambda x: reduce(*x), groupbykey(map(map_el, record_reader())))
output = list(output)
print(output)

[(0, 9), (3, 5), (2, 12), (1, 12), (4, 12)]


#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [ ]:
from typing import Iterator
from typing import Tuple
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4)

def MAP(idx, value:int):
    if idx[0] == 'mat':
        i, j = idx[1:]
        yield (i, value*vec[j])

def REDUCE(idx, products:Iterator[NamedTuple]):
    sum = 0
    for p in products:
        sum += p
    yield (idx, sum)

def RECORDREADER():
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            yield (('mat',i, j), mat[i,j])

    for i in range(vec.shape[0]):
       yield (('vac',i), vec[i])


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
print(output)

[(0, np.float64(3.0682483504343034)), (1, np.float64(3.0682483504343034)), (2, np.float64(3.0682483504343034)), (3, np.float64(3.0682483504343034)), (4, np.float64(3.0682483504343034)), ((0,), np.float64(0.530804160051104)), ((1,), np.float64(0.8018515440001659)), ((2,), np.float64(0.7963255432914828)), ((3,), np.float64(0.939267103091551))]


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [34]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [35]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for i in range(I):
        yield ((i, k), small_mat[i, j] * w)

def REDUCE(key, values):
  (i, k) = key
  yield (key, sum(values))

Проверьте своё решение

In [36]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [37]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [38]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
    for j in range(small_mat.shape[0]):
        for i in range(small_mat.shape[1]):
            yield (('small',j,i), small_mat[j,i])

    for j in range(big_mat.shape[0]):
        for k in range(big_mat.shape[1]):
            yield (('big',j,k), big_mat[j,k])

def MAP(k1, v1):
    if k1[0] == 'small':
        j, i = k1[1:]
        w = v1
        for k in range(big_mat.shape[1]):
            yield ((j, k), big_mat[i, k] * w)
    elif k1[0] == 'big':
        j, k = k1[1:]

def REDUCE(key, values):
    (i, k) = key
    yield (key, sum(values))

# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

# Verify if the solution matches the reference
print(np.allclose(reference_solution, asmatrix(solution)))  # should return true

True


Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [39]:
import numpy as np

# Matrix
rows_A = 2
cols_A = 3
cols_B = 4 * 10

# Generation
matrix_A = np.random.rand(rows_A, cols_A)
matrix_B = np.random.rand(cols_A, cols_B)

In [40]:
def flatten(nested_iterable):
    """A function for aligning nested iterable objects."""
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def group_by_key(iterable):
    grouped = {}
    for (key, value) in iterable:
        grouped[key] = grouped.get(key, []) + [value]
    return grouped.items()

def group_by_key_distributed(map_partitions, partitioner):
    """Distributed grouping by key."""
    global num_reducers
    partitions = [dict() for _ in range(num_reducers)]
    for map_partition in map_partitions:
        for (key, value) in map_partition:
            partition_id = partitions[partitioner(key)]
            partition_id[key] = partition_id.get(key, []) + [value]
    return [(i, sorted(partition.items(), key=lambda x: x[0])) for i, partition in enumerate(partitions)]

def partitioner(obj):
    """Function for determining the batch number."""
    global num_reducers
    return hash(obj) % num_reducers

def MapReduceDistributed(input_format, map_func, reduce_func, partitioner=partitioner, combiner=None):

    map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: map_func(*k1v1), record_reader)), input_format())
    if combiner is not None:
        map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: combiner(*k2v2), group_by_key(map_partition))), map_partitions)
    reduce_partitions = group_by_key_distributed(map_partitions, partitioner)
    reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: reduce_func(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

    print("{} key-value pairs were sent over a network.".format(sum(len(vs) for (k, vs) in flatten([partition for (partition_id, partition) in reduce_partitions]))))
    return reduce_outputs

def as_matrix(reduce_output):
    """Converting the reduction output to a matrix."""
    reduce_output = list(reduce_output)
    rows = max(i for ((i, k), vw) in reduce_output) + 1
    cols = max(k for ((i, k), vw) in reduce_output) + 1
    result_matrix = np.empty(shape=(rows, cols))

    for ((i, k), vw) in reduce_output:
        result_matrix[i, k] = vw

    return result_matrix


In [41]:
def input_format():
    def RECORDREADER_A(prefix):
        matrix_A_list = list()
        for j in range(matrix_A.shape[0]):
            for i in range(matrix_A.shape[1]):
                matrix_A_list.append(((prefix,j,i), matrix_A[j,i]))
        return matrix_A_list

    def RECORDREADER_B(prefix):
        matrix_B_list = list()
        for j in range(matrix_B.shape[0]):
            for i in range(matrix_B.shape[1]):
                matrix_B_list.append(((prefix,j,i), matrix_B[j,i]))
        return matrix_B_list

    yield RECORDREADER_A("A")

    yield RECORDREADER_B("B")


def map_join(key1, value1):

    matrix_id, i, j = key1
    value = value1

    if matrix_id == 'A':
        yield (j, (matrix_id, i, value))
    elif matrix_id == 'B':
        yield (i, (matrix_id, j, value))
    else: pass

def reduce_join(key, values):
    matrix_A_values = [v for v in values if v[0] == 'A']
    matrix_B_values = [v for v in values if v[0] == 'B']

    for a in matrix_A_values:
        for b in matrix_B_values:
            yield ((a[1], b[1]), a[2] * b[2])

def get_joined():
    for j in joined:
        print("Join Part:", j)
        yield j[1]

def map_multiply(key1, value1):
    yield (key1, value1)

def reduce_multiply(key, values):
    yield (key, sum(values))

num_maps = 3
num_reducers = 2

partitioned_output = MapReduceDistributed(input_format, map_join, reduce_join)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(get_joined, map_multiply, reduce_multiply)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)


solution_values = []
for p in pre_result:
    solution_values.extend(p[1])

print(solution_values)

126 key-value pairs were sent over a network.
[(0, [((0, 0), np.float64(0.3445867629616581)), ((0, 1), np.float64(0.03041774159404952)), ((0, 2), np.float64(0.3895235314990261)), ((0, 3), np.float64(0.7549847073942374)), ((0, 4), np.float64(0.3766919701695717)), ((0, 5), np.float64(0.5695978428981087)), ((0, 6), np.float64(0.00017535135728199565)), ((0, 7), np.float64(0.5610597174214389)), ((0, 8), np.float64(0.026705502558677235)), ((0, 9), np.float64(0.1543611011475926)), ((0, 10), np.float64(0.7213458911914636)), ((0, 11), np.float64(0.8622850971021838)), ((0, 12), np.float64(0.27597418569383886)), ((0, 13), np.float64(0.5400577710784273)), ((0, 14), np.float64(0.6660019698015878)), ((0, 15), np.float64(0.55142048414282)), ((0, 16), np.float64(0.24338311285315856)), ((0, 17), np.float64(0.8587946436224982)), ((0, 18), np.float64(0.7123668951318465)), ((0, 19), np.float64(0.1517066274776061)), ((0, 20), np.float64(0.17931159713112846)), ((0, 21), np.float64(0.5997372275258017)), ((0,

In [42]:
reference_solution = np.matmul(matrix_A, matrix_B)

is_close = np.allclose(reference_solution, as_matrix(solution_values))
print("Are the results close? ", is_close)

Are the results close?  True


Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [43]:
def input_format():
    def RECORDREADER_A(prefix):
        matrix_A_list = list()
        for j in range(matrix_A.shape[0]):
            for i in range(matrix_A.shape[1]):
                matrix_A_list.append(((prefix,j,i), matrix_A[j,i]))
        return matrix_A_list

    def RECORDREADER_B(prefix):
        matrix_B_list = list()
        for j in range(matrix_B.shape[0]):
            for i in range(matrix_B.shape[1]):
                matrix_B_list.append(((prefix,j,i), matrix_B[j,i]))
        return matrix_B_list

    global num_maps
    split_size = int(np.ceil(matrix_A.shape[0]*matrix_A.shape[1] / num_maps))

    for i in range(0, matrix_A.shape[0]*matrix_A.shape[1], split_size):
        yield RECORDREADER_A("A")[i:i + split_size]

    split_size = int(np.ceil(matrix_B.shape[0]*matrix_B.shape[1] / num_maps))

    for i in range(0, matrix_B.shape[0]*matrix_B.shape[1], split_size):
        yield RECORDREADER_B("B")[i:i + split_size]


def map_join(key1, value1):

    matrix_id, i, j = key1
    value = value1

    if matrix_id == 'A':
        yield (j, (matrix_id, i, value))
    elif matrix_id == 'B':
        yield (i, (matrix_id, j, value))
    else: pass

def reduce_join(key, values):
    matrix_A_values = [v for v in values if v[0] == 'A']
    matrix_B_values = [v for v in values if v[0] == 'B']

    for a in matrix_A_values:
        for b in matrix_B_values:
            yield ((a[1], b[1]), a[2] * b[2])

def get_joined():
    for j in joined:
        print("Join Part:", j)
        yield j[1]

def map_multiply(key1, value1):
    yield (key1, value1)

def reduce_multiply(key, values):
    yield (key, sum(values))

num_maps = 3
num_reducers = 2

partitioned_output = MapReduceDistributed(input_format, map_join, reduce_join)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(get_joined, map_multiply, reduce_multiply)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)


solution_values = []
for p in pre_result:
    solution_values.extend(p[1])

print(solution_values)

126 key-value pairs were sent over a network.
[(0, [((0, 0), np.float64(0.3445867629616581)), ((0, 1), np.float64(0.03041774159404952)), ((0, 2), np.float64(0.3895235314990261)), ((0, 3), np.float64(0.7549847073942374)), ((0, 4), np.float64(0.3766919701695717)), ((0, 5), np.float64(0.5695978428981087)), ((0, 6), np.float64(0.00017535135728199565)), ((0, 7), np.float64(0.5610597174214389)), ((0, 8), np.float64(0.026705502558677235)), ((0, 9), np.float64(0.1543611011475926)), ((0, 10), np.float64(0.7213458911914636)), ((0, 11), np.float64(0.8622850971021838)), ((0, 12), np.float64(0.27597418569383886)), ((0, 13), np.float64(0.5400577710784273)), ((0, 14), np.float64(0.6660019698015878)), ((0, 15), np.float64(0.55142048414282)), ((0, 16), np.float64(0.24338311285315856)), ((0, 17), np.float64(0.8587946436224982)), ((0, 18), np.float64(0.7123668951318465)), ((0, 19), np.float64(0.1517066274776061)), ((0, 20), np.float64(0.17931159713112846)), ((0, 21), np.float64(0.5997372275258017)), ((0,

In [44]:
reference_solution = np.matmul(matrix_A, matrix_B)

is_close = np.allclose(reference_solution, as_matrix(solution_values))
print("Are the results close? ", is_close)

Are the results close?  True
